<a href="https://colab.research.google.com/github/SteveChengChen/data-analysis/blob/main/STAT_5243_HW1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import time

# 1. Basic logistic regression utilities
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def neg_log_likelihood(beta, X, y):

    p = sigmoid(X @ beta)

    eps = 1e-15
    log_likelihood = np.sum( y * np.log(p + eps) + (1 - y)*np.log(1 - p + eps) )
    return -log_likelihood

def grad_neg_log_likelihood(beta, X, y):

    p = sigmoid(X @ beta)
    grad = X.T @ (p - y)
    return grad

def hess_neg_log_likelihood(beta, X, y):

    p = sigmoid(X @ beta)

    W = p * (1.0 - p)
    sqrtW = np.sqrt(W)
    X_weighted = X * sqrtW[:, np.newaxis]
    H = X_weighted.T @ X_weighted
    return H

# 2. Gradient Descent and Newton's Method

def logistic_regression_gd(X, y, alpha=1e-3, tol=1e-6, max_iter=1000):

    N, D = X.shape
    beta = np.zeros(D)
    obj_history = []

    for it in range(max_iter):
        grad = grad_neg_log_likelihood(beta, X, y)
        beta_new = beta - alpha * grad

        if np.linalg.norm(beta_new - beta) < tol:
            beta = beta_new
            break

        beta = beta_new
        obj_history.append(neg_log_likelihood(beta, X, y))

    return beta, it+1, obj_history

def logistic_regression_newton(X, y, tol=1e-9, max_iter=100):

    N, D = X.shape
    beta = np.zeros(D)
    obj_history = []

    for it in range(max_iter):
        grad = grad_neg_log_likelihood(beta, X, y)
        H = hess_neg_log_likelihood(beta, X, y)

        H_reg = H + 1e-8 * np.eye(D)
        delta = np.linalg.solve(H_reg, -grad)

        beta_new = beta + delta
        if np.linalg.norm(delta) < tol:
            beta = beta_new
            break

        beta = beta_new
        obj_history.append(neg_log_likelihood(beta, X, y))

    return beta, it+1, obj_history


# 3. Compare two scenarios

def generate_synthetic_logistic_data(N, D, seed=42):

    np.random.seed(seed)
    X = np.random.randn(N, D)
    true_beta = np.random.randn(D)
    logits = X @ true_beta
    probs = sigmoid(logits)
    y = (np.random.rand(N) < probs).astype(float)
    return X, y

def run_experiment(N, D):

    print(f"\n Experiment with N={N}, D={D} ")
    X, y = generate_synthetic_logistic_data(N, D)

    start_gd = time.time()
    beta_gd, iters_gd, obj_hist_gd = logistic_regression_gd(X, y)
    time_gd = time.time() - start_gd

    start_newton = time.time()
    beta_newton, iters_newton, obj_hist_newton = logistic_regression_newton(X, y)
    time_newton = time.time() - start_newton

    print(f"Gradient Descent:   iterations={iters_gd}, total_time={time_gd:.3f} s")
    print(f"Newton's Method:    iterations={iters_newton}, total_time={time_newton:.3f} s")

    nll_gd = neg_log_likelihood(beta_gd, X, y)
    nll_newton = neg_log_likelihood(beta_newton, X, y)
    print(f"Final NLL: GD={nll_gd:.4f}  vs.  Newton={nll_newton:.4f}")

# 4. Main demo
if __name__ == "__main__":
    run_experiment(N=10000, D=1000)

    run_experiment(N=1000, D=10)



 Experiment with N=10000, D=1000 


<ipython-input-2-d7951525f95a>:6: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


Gradient Descent:   iterations=1000, total_time=5.045 s
Newton's Method:    iterations=100, total_time=11.635 s
Final NLL: GD=89.7977  vs.  Newton=0.0000

 Experiment with N=1000, D=10 
Gradient Descent:   iterations=292, total_time=0.034 s
Newton's Method:    iterations=7, total_time=0.002 s
Final NLL: GD=385.3359  vs.  Newton=385.3359


##### Choose a relatively large number of samples N and feature dimension D
##### N=10000, D=1000
##### Run Gradient Descent(1st approx) versus Newton's Method(2nd approx)
##### Gradient Descent took 1000 iterations in 5.045 s
##### Newton's Method took 100 iterations in 11.635 s
##### the total runtime for 2nd order was slower, suggests that Hessian construction/inversion cost can dominate in large-scale problems
##### Choose a smaller problem, N=1000, D=10
##### Newton's Method converged in just 7 iterations(about 0.002 s), naive Gradient Descent required 292 iterations (about 0.034 s)
##### In smaller, well-conditioned problems, Newton converges in very few steps, outpacing simpler gradient methods
